In [36]:
django-admin startproject csv_analysis_project
cd csv_analysis_project && django-admin startapp analysis


CommandError: 'C:\Users\AMRIT HONEY\OneDrive\Desktop\python project\csv_analysis_project' already exists
CommandError: 'C:\Users\AMRIT HONEY\OneDrive\Desktop\python project\csv_analysis_project\analysis' already exists


In [37]:
# Define the path to settings.py
settings_path = 'csv_analysis_project/csv_analysis_project/settings.py'

# Read the current settings file
with open(settings_path, 'r') as file:
    settings_lines = file.readlines()

# Prepare new settings to add
new_settings = [
    "\n# Installed Apps\n",
    "INSTALLED_APPS += ['analysis']\n",
    "\n# Media files settings\n",
    "MEDIA_URL = '/media/'\n",
    "MEDIA_ROOT = BASE_DIR / 'media'\n"
]

# Find the index to insert the new settings
insert_index = next((i for i, line in enumerate(settings_lines) if line.startswith("INSTALLED_APPS =")), len(settings_lines))

# Insert new settings into the file
updated_settings = settings_lines[:insert_index + 1] + new_settings + settings_lines[insert_index + 1:]

# Write the updated settings back to the file
with open(settings_path, 'w') as file:
    file.writelines(updated_settings)


In [38]:
# Define the path to forms.py
forms_path = 'csv_analysis_project/analysis/forms.py'

# Create forms.py with the form for file uploads
with open(forms_path, 'w') as file:
    file.write("""
from django import forms

class UploadFileForm(forms.Form):
    file = forms.FileField()
""")


In [39]:
# Define the path to views.py
views_path = 'csv_analysis_project/analysis/views.py'

# Create views.py with view functions
with open(views_path, 'w') as file:
    file.write("""
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from django.shortcuts import render
from django.core.files.storage import default_storage
from .forms import UploadFileForm
from django.http import HttpResponse
from io import BytesIO
import base64

def upload_file(request):
    if request.method == 'POST':
        form = UploadFileForm(request.POST, request.FILES)
        if form.is_valid():
            file = request.FILES['file']
            file_path = default_storage.save(file.name, file)
            file_url = default_storage.url(file_path)
            return render(request, 'analysis/results.html', {'file_url': file_url})
    else:
        form = UploadFileForm()
    return render(request, 'analysis/upload.html', {'form': form})

def results(request):
    file_url = request.GET.get('file_url')
    if file_url:
        file_path = default_storage.path(file_url)
        df = pd.read_csv(file_path)
        
        # Basic data analysis
        summary = df.describe()
        missing_values = df.isnull().sum()
        
        # Generate plots
        hist_plots = []
        for column in df.select_dtypes(include=['number']).columns:
            fig, ax = plt.subplots()
            df[column].plot(kind='hist', ax=ax, bins=20)
            plt.title(f'Histogram of {column}')
            
            # Save plot to a BytesIO object
            img = BytesIO()
            plt.savefig(img, format='png')
            img.seek(0)
            plot_url = 'data:image/png;base64,' + base64.b64encode(img.getvalue()).decode('utf8')
            hist_plots.append((column, plot_url))

        context = {
            'summary': summary.to_html(),
            'missing_values': missing_values.to_frame().to_html(),
            'hist_plots': hist_plots,
        }

        return render(request, 'analysis/results.html', context)
    return HttpResponse("No file uploaded.")
""")


In [40]:
# Define the path to urls.py
urls_path = 'csv_analysis_project/csv_analysis_project/urls.py'

# Read the current urls.py file
with open(urls_path, 'r') as file:
    urls_lines = file.readlines()

# Prepare new URL patterns
new_urls = [
    "from analysis import views\n",
    "from django.conf import settings\n",
    "from django.conf.urls.static import static\n",
    "\n",
    "urlpatterns += [\n",
    "    path('', views.upload_file, name='upload_file'),\n",
    "    path('results/', views.results, name='results'),\n",
    "] + static(settings.MEDIA_URL, document_root=settings.MEDIA_ROOT)\n"
]

# Find the index to insert the new URL patterns
insert_index = next((i for i, line in enumerate(urls_lines) if line.startswith("urlpatterns =")), len(urls_lines))

# Insert new URLs into the file
updated_urls = urls_lines[:insert_index + 1] + new_urls + urls_lines[insert_index + 1:]

# Write the updated URLs back to the file
with open(urls_path, 'w') as file:
    file.writelines(updated_urls)


In [41]:
import os

# Paths for Django project
project_dir = 'csv_analysis_project'
app_dir = os.path.join(project_dir, 'analysis')
templates_dir = os.path.join(app_dir, 'templates', 'analysis')
settings_path = os.path.join(project_dir, 'csv_analysis_project', 'settings.py')
views_path = os.path.join(app_dir, 'views.py')
forms_path = os.path.join(app_dir, 'forms.py')
urls_path = os.path.join(project_dir, 'csv_analysis_project', 'urls.py')

# Create necessary directories
os.makedirs(templates_dir, exist_ok=True)

# Write forms.py
with open(forms_path, 'w') as file:
    file.write("""
from django import forms

class UploadFileForm(forms.Form):
    file = forms.FileField()
""")

# Write views.py
with open(views_path, 'w') as file:
    file.write("""
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from django.shortcuts import render
from django.core.files.storage import default_storage
from .forms import UploadFileForm
from django.http import HttpResponse
from io import BytesIO
import base64

def upload_file(request):
    if request.method == 'POST':
        form = UploadFileForm(request.POST, request.FILES)
        if form.is_valid():
            file = request.FILES['file']
            file_path = default_storage.save(file.name, file)
            file_url = default_storage.url(file_path)
            return render(request, 'analysis/results.html', {'file_url': file_url})
    else:
        form = UploadFileForm()
    return render(request, 'analysis/upload.html', {'form': form})

def results(request):
    file_url = request.GET.get('file_url')
    if file_url:
        file_path = default_storage.path(file_url)
        df = pd.read_csv(file_path)
        
        # Basic data analysis
        summary = df.describe()
        missing_values = df.isnull().sum()
        
        # Generate plots
        hist_plots = []
        for column in df.select_dtypes(include=['number']).columns:
            fig, ax = plt.subplots()
            df[column].plot(kind='hist', ax=ax, bins=20)
            plt.title(f'Histogram of {column}')
            
            # Save plot to a BytesIO object
            img = BytesIO()
            plt.savefig(img, format='png')
            img.seek(0)
            plot_url = 'data:image/png;base64,' + base64.b64encode(img.getvalue()).decode('utf8')
            hist_plots.append((column, plot_url))

        context = {
            'summary': summary.to_html(),
            'missing_values': missing_values.to_frame().to_html(),
            'hist_plots': hist_plots,
        }

        return render(request, 'analysis/results.html', context)
    return HttpResponse("No file uploaded.")
""")

# Write urls.py
with open(urls_path, 'r') as file:
    urls_lines = file.readlines()

new_urls = [
    "from analysis import views\n",
    "from django.conf import settings\n",
    "from django.conf.urls.static import static\n",
    "\n",
    "urlpatterns += [\n",
    "    path('', views.upload_file, name='upload_file'),\n",
    "    path('results/', views.results, name='results'),\n",
    "] + static(settings.MEDIA_URL, document_root=settings.MEDIA_ROOT)\n"
]

insert_index = next((i for i, line in enumerate(urls_lines) if line.startswith("urlpatterns =")), len(urls_lines))

updated_urls = urls_lines[:insert_index + 1] + new_urls + urls_lines[insert_index + 1:]

with open(urls_path, 'w') as file:
    file.writelines(updated_urls)

# Write upload.html
upload_template_path = os.path.join(templates_dir, 'upload.html')
with open(upload_template_path, 'w') as file:
    file.write("""
<!DOCTYPE html>
<html>
<head>
    <title>Upload CSV</title>
</head>
<body>
    <h1>Upload CSV File</h1>
    <form method="post" enctype="multipart/form-data">
        {% csrf_token %}
        {{ form.as_p }}
        <button type="submit">Upload</button>
    </form>
</body>
</html>
""")

# Write results.html
results_template_path = os.path.join(templates_dir, 'results.html')
with open(results_template_path, 'w') as file:
    file.write("""
<!DOCTYPE html>
<html>
<head>
    <title>Analysis Results</title>
</head>
<body>
    <h1>Analysis Results</h1>
    
    <h2>Summary Statistics</h2>
    <div>{{ summary|safe }}</div>
    
    <h2>Missing Values</h2>
    <div>{{ missing_values|safe }}</div>
    
    <h2>Histograms</h2>
    {% for column, plot_url in hist_plots %}
        <h3>Histogram for {{ column }}</h3>
        <img src="{{ plot_url }}" alt="Histogram for {{ column }}">
    {% endfor %}
    
</body>
</html>
""")


In [44]:
!csv_analysis_project
python manage.py migrate
python manage.py runserver


SyntaxError: invalid syntax (2515227821.py, line 2)